In [1]:
import sys
sys.path.append('../')
import utils
import models
import numpy as np 
import matplotlib.pyplot as plt 
import cv2
import tensorflow as tf 
 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

model, x_test, y_test = models.train(epochs=1000, batch_size=16)

l_loss: 0.0158 - val_acc: 0.5798
Epoch 852/1000
15/15 [==============================] - 58s 4s/step - loss: 0.0167 - acc: 0.6639 - val_loss: 0.0222 - val_acc: 0.5654
Epoch 853/1000
15/15 [==============================] - 60s 4s/step - loss: 0.0133 - acc: 0.6226 - val_loss: 0.0168 - val_acc: 0.6378
Epoch 854/1000
15/15 [==============================] - 55s 4s/step - loss: 0.0160 - acc: 0.6604 - val_loss: 0.0125 - val_acc: 0.7142
Epoch 855/1000
15/15 [==============================] - 62s 4s/step - loss: 0.0166 - acc: 0.6575 - val_loss: 0.0214 - val_acc: 0.6768
Epoch 856/1000
15/15 [==============================] - 62s 4s/step - loss: 0.0164 - acc: 0.6223 - val_loss: 0.0200 - val_acc: 0.6282
Epoch 857/1000
15/15 [==============================] - 59s 4s/step - loss: 0.0136 - acc: 0.6122 - val_loss: 0.0156 - val_acc: 0.6595
Epoch 858/1000
15/15 [==============================] - 63s 4s/step - loss: 0.0155 - acc: 0.6233 - val_loss: 0.0247 - val_acc: 0.6894
Epoch 859/1000
15/15 [=======